<a href="https://colab.research.google.com/github/nbartulic/Testes_Dio/blob/main/Teste_Nath_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Projeto de Transfer Learning: Cats vs Dogs
# Autor: Nathália
# Data: 2024.12.06
# Descrição: Este projeto utiliza Transfer Learning para classificar imagens de gatos e cachorros com o dataset Cats vs Dogs da TensorFlow (Teste DIO)

# ==========================
# 1. Importação das Bibliotecas
# ==========================
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# ==========================
# 2. Carregamento do Dataset
# ==========================
# Baixar o dataset Cats vs Dogs da TensorFlow
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
train_val_dataset = dataset['train']

# Dividir em treino (80%) e validação (20%)
train_dataset = train_val_dataset.take(20000)
val_dataset = train_val_dataset.skip(20000)

# ==========================
# 3. Pré-processamento dos Dados
# ==========================
# Função para redimensionar e normalizar as imagens
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0
    return image, label

# Aplicar a função de pré-processamento
train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(32)
val_dataset = val_dataset.map(preprocess).batch(32)

# ==========================
# 4. Construção do Modelo
# ==========================
# Carregar MobileNetV2 como modelo base
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Adicionar camadas personalizadas
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Saída binária: 1 para cachorro, 0 para gato
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ==========================
# 5. Treinamento do Modelo
# ==========================
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

# ==========================
# 6. Avaliação do Modelo
# ==========================
# Exibir gráficos de acurácia e perda
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Acurácia Treinamento')
plt.plot(val_acc, label='Acurácia Validação')
plt.legend(loc='lower right')
plt.title('Acurácia')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Perda Treinamento')
plt.plot(val_loss, label='Perda Validação')
plt.legend(loc='upper right')
plt.title('Perda')
plt.show()

# ==========================
# 7. Salvar o Modelo Treinado
# ==========================
model.save('cats_vs_dogs_model.h5')

Epoch 1/10
505/625 ━━━━━━━━━━━━━━━━━━━━ 50s 422ms/step - accuracy: 0.9366 - loss: 0.1589